## Libraries

In [1]:
import os
import time

import polars as pl
import pandas as pd
import duckdb

# import pyspark
# from pyspark.sql import SparkSession

#Might add ibis vaex

## Raw Files [Path]

In [2]:
raw_main_data = r'Data_extracted\creditcard.csv'

## File reading

In [3]:
def read_files(path: str, reader: str) -> dict:
    finalTime = None
    df = None
    if reader  == 'duckDB':
        
        startTime = time.time()

        con = duckdb.connect('Files/my_database.duckdb')
        con.execute("""
            CREATE TABLE my_table
            SELECT * FROM read_csv_auto('Data_extracted\creditcard.csv');
        """)
        df = con.execute("SELECT * FROM my_table LIMIT 5").fetchdf()
        con.close()

        endTime = time.time()

    elif reader  == 'pandas':
        startTime = time.time()
        df = pd.read_csv(path)
        endTime = time.time()
    elif reader  == 'polars':
        startTime = time.time()
        df = pl.read_csv(path, ignore_errors=True)
        endTime = time.time()

    finalTime = endTime - startTime
    assert len(df) > 0
    
    return {
            'reader':reader,
            'reading_time': finalTime,
            'data': df
            }

def data_populating(df, reader: str, register_multipliyer: int) -> dict:
    finalTime = None
    df_concated = None
    if reader == 'duckDB':
        con = duckdb.connect('Files/my_database.duckdb')
        startTime = time.time()
        con.register("my_data", df)
        for _ in range(register_multipliyer):
            con.execute("INSERT INTO my_table SELECT * FROM my_data")
        result = duckdb.sql("SELECT * FROM my_table").to_df()
        endTime = time.time()
        con.close()
    elif reader == 'pandas':
        startTime = time.time()
        df_concated = pd.concat([df]*3, ignore_index=True)
        endTime = time.time()

    elif reader == 'polars':
        startTime = time.time()
        # for i in range(register_multipliyer):
        #     df_concated = df.vstack(f'{saving_path}_polars.parquet')
        df_concated = pl.concat([df] * register_multipliyer)

        endTime = time.time()

    
    finalTime = endTime - startTime

    assert len(df_concated) == register_multipliyer*len(df)
    
    return {
            'reader':reader,
            'reading_time': finalTime,
            }

def format_to_parquet(df, reader: str, saving_path: str):
    finalTime = None
    if 'duckDB':
        startTime = time.time()
        df.to_parquet()
        endTime = time.time()
    elif 'pandas':
        df.to_arrow(f'{saving_path}_polars.parquet')
    elif 'polars':
        df.write_parquet(f'{saving_path}_polars.parquet')
    
    finalTime = endTime - startTime

    return {
            'reader':reader,
            'reading_time': finalTime,
            }

In [4]:
a = read_files(path = raw_main_data,reader = 'duckDB')
data_populating(df = pd.read_csv('Data_extracted\creditcard.csv'), reader = 'duckDB', register_multipliyer = 2)

ParserException: Parser Error: syntax error at or near "SELECT"

## Reading Speed

## Register Increment Speed

## Data Grouping Speed

## Data Filtering Speed

## Column Sorting Speed

## Memory Usage

## Data Offload Speed